---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
#!cat university_towns.txt

In [4]:
a_string = 'this is a string (in python) with brackets [like this one]'

In [5]:
a_string.partition('[')

('this is a string (in python) with brackets ', '[', 'like this one]')

In [6]:
a_string.split('[')

['this is a string (in python) with brackets ', 'like this one]']

In [7]:
#!cat gdplev.xls

In [8]:
#test=pd.read_fwf('university_towns.txt', header=None, names=['ST'])
#test.ST.apply(lambda x: re.search('(.+)\[edit\]',x))

In [9]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'.'''
    
    with open('university_towns.txt', mode='r+' , encoding='utf8') as f:
        data = []
        
        for line in f:
            m = re.search('(.+)\[edit\]',line) #get state
            if m:
                state=m.group(1)
            else:
                town = line.split('(')[0].strip()
                data.append([state,town])
    
    unitowns = pd.DataFrame(data,columns=["State","RegionName"])

    return unitowns

get_list_of_university_towns().head(10)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [10]:
def fx(x1,x2,x3):
    if x2<x1 and x3<x2:
        return 'recession'
    else:
        return 'other'

In [11]:
def fre(x1,x2,x3):
    if x2>x1 and x3>x2:
        return 'recessionEnds'
    else:
        return 'other'

In [12]:
#test['Difference']=test['GDP'].diff(periods=1)

In [13]:
#test['Sign']=test['Difference'].apply(lambda x: 'positive' if x>=0 else 'negative')

In [14]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    test = pd.read_excel('gdplev.xls', header=None, skiprows=8, parse_cols =[4,5], names=['Quarter', 'GDP'])
    test['GDP_II']=test['GDP'].shift(periods=-1)
    test['GDP_III']=test['GDP'].shift(periods=-2)
    test['GDP_IV']=test['GDP'].shift(periods=-3)
    test['GDP_V']=test['GDP'].shift(periods=-4)
    test['RecessionStarts']=np.vectorize(fx)(test['GDP'], test['GDP_II'],test['GDP_III'])
    test=test.loc[(test['RecessionStarts']=='recession') & (test['Quarter']>='2000q1')]
    
    return test['Quarter'].iloc[0]

In [15]:
get_recession_start()

'2008q3'

In [16]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    test = pd.read_excel('gdplev.xls', header=None, skiprows=8, parse_cols =[4,5], names=['Quarter', 'GDP'])
    test['GDP_II']=test['GDP'].shift(periods=-1)
    test['GDP_III']=test['GDP'].shift(periods=-2)
    test['GDP_IV']=test['GDP'].shift(periods=-3)
    test['GDP_V']=test['GDP'].shift(periods=-4)
    test=test.loc[test['Quarter']>=get_recession_start()]
    test['RecessionEnds']=np.vectorize(fre)(test['GDP'], test['GDP_II'],test['GDP_III'])
    #test['RecessionEndsII']=test['RecessionEnds'].shift(-1)
    test=test.loc[(test['RecessionEnds']=='recessionEnds') ]  
    return test['Quarter'].iloc[2]

In [17]:
get_recession_end()

'2009q4'

In [18]:
test = pd.read_excel('gdplev.xls', header=None, skiprows=8, parse_cols =[4,5], names=['Quarter', 'GDP'])

In [19]:
test.loc[(test['Quarter']>=get_recession_start()) & (test['Quarter']<=get_recession_end())]['GDP'].argmin()

249

In [20]:
test.Quarter.iloc[test.loc[(test['Quarter']>=get_recession_start()) & (test['Quarter']<=get_recession_end())]['GDP'].argmin()]

'2009q2'

In [21]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    test = pd.read_excel('gdplev.xls', header=None, skiprows=8, parse_cols =[4,5], names=['Quarter', 'GDP'])
    ans=test.Quarter.iloc[test.loc[(test['Quarter']>=get_recession_start()) & (test['Quarter']<=get_recession_end())]['GDP'].argmin()]
    return ans

In [22]:
get_recession_bottom()

'2009q2'

In [23]:
df=pd.read_csv('City_Zhvi_AllHomes.csv')
df['State']=df['State'].map(states)
colnames = df.columns
lst1 = ['State', 'RegionName']
#lst1=list(colnames[1:3])
lst2=list(colnames[list(colnames).index('2000-01'):])
keepcols=lst1+lst2
df=df[keepcols]
dfmelt=pd.melt(df, id_vars=lst1, var_name="Date", value_name="Value")
dfmelt['Date'] = pd.to_datetime(dfmelt['Date'])
dfmelt['Year'] = pd.DatetimeIndex(dfmelt['Date']).year
dfmelt['Quarter'] = pd.DatetimeIndex(dfmelt['Date']).quarter
dfmelt['q'] = 'q'
dfmelt['Date'] = dfmelt['Year'].map(str)+dfmelt['q']+dfmelt['Quarter'].map(str)
dfmelt.drop(['Year', 'Quarter', 'q'], axis=1, inplace=True)
dfgroup=dfmelt.groupby(['State', 'RegionName', 'Date']).mean()
dfgroup.reset_index(inplace=True)
ptable=pd.pivot_table(dfgroup, index=['State', 'RegionName'], columns=['Date'], values='Value')
dummytable=df[['State', 'RegionName', '2000-01']].set_index(['State', 'RegionName'])
ans=pd.merge(dummytable, ptable, how='left',  left_index=['State', 'RegionName'], right_index=['State', 'RegionName'] )
ans.drop(['2000-01'], inplace=True, axis=1)

In [24]:
dummytable=df[['State', 'RegionName', '2000-01']].set_index(['State', 'RegionName'])

In [25]:
ans=pd.merge(dummytable, ptable, how='left',  left_index=['State', 'RegionName'], right_index=['State', 'RegionName'] )

In [26]:
ans.drop('2000-01', inplace=True)

In [27]:
dfgroup=dfmelt.groupby(['State', 'RegionName', 'Date']).mean()

In [28]:
dfgroup.head()

Value
State   RegionName Date                
Alabama Adamsville 2000q1  69033.333333
                   2000q2  69166.666667
                   2000q3  69800.000000
                   2000q4  71966.666667
                   2001q1  73466.666667

In [29]:
dfgroup.reset_index(inplace=True)

In [30]:
dfgroup.head()

,State,RegionName,Date,Value
0,Alabama,Adamsville,2000q1,69033.333333
1,Alabama,Adamsville,2000q2,69166.666667
2,Alabama,Adamsville,2000q3,69800.000000
3,Alabama,Adamsville,2000q4,71966.666667
4,Alabama,Adamsville,2001q1,73466.666667


In [31]:
dfgroup.shape

(709664, 4)

In [32]:
#dfgroup.pivot(columns='Date', index='State', values='Value')

In [33]:
#colnames = df.columns

In [34]:
#[i for i,x in enumerate(colnames) if x=='2000-01']

In [35]:
#list(colnames).index('2000-01')

In [36]:
#pd.to_datetime(colnames[6:]).quarter

In [37]:
#datetimelist=[]
#for i in colnames[6:]:
#    print(str(pd.to_datetime(i).year)+'q'+str(pd.to_datetime(i).quarter))
#    datetimelist.append(str(pd.to_datetime(i).year)+'q'+str(pd.to_datetime(i).quarter))

In [38]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State']=df['State'].map(states)
    colnames = df.columns
    lst1 = ['State', 'RegionName']
    #lst1=list(colnames[1:3])
    lst2=list(colnames[list(colnames).index('2000-01'):])
    keepcols=lst1+lst2
    df=df[keepcols]
    dfmelt=pd.melt(df, id_vars=lst1, var_name="Date", value_name="Value")
    dfmelt['Date'] = pd.to_datetime(dfmelt['Date'])
    dfmelt['Year'] = pd.DatetimeIndex(dfmelt['Date']).year
    dfmelt['Quarter'] = pd.DatetimeIndex(dfmelt['Date']).quarter
    dfmelt['q'] = 'q'
    dfmelt['Date'] = dfmelt['Year'].map(str)+dfmelt['q']+dfmelt['Quarter'].map(str)
    dfmelt.drop(['Year', 'Quarter', 'q'], axis=1, inplace=True)
    dfgroup=dfmelt.groupby(['State', 'RegionName', 'Date']).mean()
    dfgroup.reset_index(inplace=True)
    ptable=pd.pivot_table(dfgroup, index=['State', 'RegionName'], columns=['Date'], values='Value')
    dummytable=df[['State', 'RegionName', '2000-01']].set_index(['State', 'RegionName'])
    ans=pd.merge(dummytable, ptable, how='left',  left_index=['State', 'RegionName'], right_index=['State', 'RegionName'] )
    ans.drop(['2000-01'], inplace=True, axis=1)
    return ans

In [39]:
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

In [59]:
unitowns = get_list_of_university_towns()
bottom = get_recession_bottom()
start = get_recession_start()
hdata = convert_housing_data_to_quarters()
hdata['ratio'] = hdata[bottom] - hdata[start]
hdata = hdata[[bottom,start,'ratio']]


In [60]:
uni_town = set(unitowns['RegionName'])
hdata=hdata.reset_index()

hdata.head()

,State,RegionName,2009q2,2008q3,ratio
0,Alabama,Adamsville,85766.666667,84866.666667,900.000000
1,Alabama,Alabaster,149666.666667,155333.333333,-5666.666667
2,Alabama,Albertville,84333.333333,85333.333333,-1000.000000
3,Alabama,Arab,115366.666667,108933.333333,6433.333333
4,Alabama,Ardmore,143900.000000,149100.000000,-5200.000000


In [62]:
def is_uni_town(row):
        #check if the town is a university towns or not.
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
hdata['is_uni'] = hdata.apply(is_uni_town, axis=1)

In [58]:
hdata.head()

,State,RegionName,2009q2,2008q3,ratio,is_uni
0,Alabama,Adamsville,85766.666667,84866.666667,900.000000,0
1,Alabama,Alabaster,149666.666667,155333.333333,-5666.666667,0
2,Alabama,Albertville,84333.333333,85333.333333,-1000.000000,0
3,Alabama,Arab,115366.666667,108933.333333,6433.333333,0
4,Alabama,Ardmore,143900.000000,149100.000000,-5200.000000,0


In [69]:
nu=hdata.loc[hdata['is_uni']==0]['ratio'].dropna()
un=hdata.loc[hdata['is_uni']==1]['ratio'].dropna()

In [70]:
nu.mean()

-13287.494550806408

In [71]:
un.mean()

-9350.3904644471841

In [78]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    hdata['ratio'] = hdata[bottom] - hdata[start]
    hdata = hdata[[bottom,start,'ratio']]
    uni_town = set(unitowns['RegionName'])
    hdata=hdata.reset_index()
    hdata['is_uni'] = hdata.apply(is_uni_town, axis=1)
    nu=hdata.loc[hdata['is_uni']==0]['ratio'].dropna()
    un=hdata.loc[hdata['is_uni']==1]['ratio'].dropna()
    p_val = list(ttest_ind(nu, un))[1]
    result = (True,p_val,'university town')
    return result

In [79]:
run_ttest()

(True, 7.5252146217457879e-06, 'university town')